<a href="https://colab.research.google.com/github/ransjnr/building-ai-agents-with-autogen/blob/main/Calculator_tool_to_math_tutor_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyautogen
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.1 MB/s eta 0:00:00


In [ ]:
##import requried libraries

import os
from autogen import ConversableAgent, UserProxyAgent, AssistantAgent
from openai import AzureOpenAI
from google.colab import userdata


In [ ]:
endpoint = userdata.get("AZURE_OPENAI_ENDPOINT")
model_name = "o4-mini"
deployment = "o4-mini-deployment"

subscription_key = userdata.get("AZURE_OPENAI_KEY")
api_version = "2024-12-01-preview"

In [ ]:
llm_config = {
    "config_list": [
        {
            "model": deployment,
            "base_url": endpoint,
            "api_type": "azure",
            "api_key": subscription_key,
            "api_version": api_version,
        }
    ]
}

###Defining a calculator tool

In [ ]:
from typing import Annotated, Literal

Operator = Literal["+", "-", "*", "/"]

def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
  if operator == "+":
    return a + b
  elif operator == "-":
    return a - b
  elif operator == "*":
    return a * b
  elif operator == "/":
    return int(a / b)
  else:
    raise ValueError("Invalid operator")

###Defining a Math tutor Agent

In [ ]:
assistant = ConversableAgent(
    name = "Assistant",
    system_message="You are a helpful AI assistant. "
                   "You can help with simple calculations. "
                   "Return 'TERMINATE' when the task is done.",
    llm_config=llm_config,
)

###A user proxy to interact

In [ ]:
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

###Register the tools with the agents

In [ ]:
assistant.register_for_llm(name="calculator", description="A simple calculator")(calculator)

user_proxy.register_for_execution(name="claculator")(calculator)

[autogen.oai.client: 05-13 16:41:34] {353} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


###Alternative way for registering tools

In [ ]:
from autogen import register_function
#register the calculator function to the two agents.
register_function(
    calculator,
    caller=assistant,
    executor=user_proxy,
    name="calculator",
    description="A simple calculator",
)

[autogen.oai.client: 05-13 16:45:34] {353} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


/usr/local/lib/python3.11/dist-packages/autogen/agentchat/conversable_agent.py:3424: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)


###Initiate the conversation

In [ ]:
chat_result = user_proxy.initiate_chat(assistant, message="What is (44232 + 13312 / (232 - 32)) * 5?")

User (to Assistant):

What is (44232 + 13312 / (232 - 32)) * 5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
[autogen.oai.client: 05-13 16:50:55] {696} WARNING - Model o4-mini is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Assistant (to User):

221492.8

TERMINATE

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (0e42e3ef-6882-451a-bb7a-de14907874bf): Termination message condition on agent 'User' met
